In [51]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate


from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

# LLM 초기화
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
)

# 캐시 디렉토리 설정
cache_dir = LocalFileStore("./.cache/")

# 텍스트 스플리터 설정
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

# URL에서 문서 로드
loader = WebBaseLoader("https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223")
docs = loader.load_and_split(text_splitter=splitter)

# cached_embeddings 설정
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# vectorstore 생성
vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever()

# ConversationBufferMemory 초기화
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history"
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]

# Stuff Documents prompt template
stuff_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 제공된 컨텍스트를 기반으로 질문에 답하는 도움이 되는 어시스턴트입니다.
    다음 컨텍스트 조각들을 사용하여 사용자의 질문에 답하세요.
    컨텍스트를 기반으로 답을 모르는 경우, 모른다고 말하세요.
    답을 지어내려고 하지 마세요.
    
    컨텍스트:
    {context}
    """),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

# Stuff Documents Chain with memory
def stuff_chain_with_memory(question):
    # search relevant documents
    docs = retriever.get_relevant_documents(question)
    
    # format docs
    context = format_docs(docs)
    
    # load chat history
    chat_history = load_memory(None)
    
    # create chain input
    chain_input = {
        "context": context,
        "chat_history": chat_history,
        "question": question
    }
    
    # generate response
    response = llm.invoke(stuff_prompt.format_messages(**chain_input))
    
    # save to memory
    memory.save_context(
        {"input": question},
        {"output": response.content}
    )
    
    return response

chain = (
    {
        "context": retriever | format_docs,
        "chat_history": RunnableLambda(load_memory),
        "question": RunnablePassthrough()
    }
    | stuff_prompt
    | llm
)

def invoke_chain_with_memory(question):
    response = chain.invoke(question)
    memory.save_context(
        {"input": question},
        {"output": response.content}
    )
    print(f"Q: {question}")
    print(f"A: {response.content}")
    print("-" * 50)
    return response

# test RAG pipeline with questions
print("=== Stuff Documents Chain RAG pipeline with memory ===\n")

# clear memory
memory.clear()

# 질문 1: Aaronson이 유죄인가?
print("1. Aaronson이 유죄인가?")
invoke_chain_with_memory("Aaronson은 유죄인가요?")

# 질문 2: 그가 테이블에 어떤 메시지를 썼나?
print("2. 그가 테이블에 어떤 메시지를 썼나요?")
invoke_chain_with_memory("그가 테이블에 어떤 메시지를 썼나요?")

# 질문 3: Julia는 누구인가?
print("3. Julia는 누구인가요?")
invoke_chain_with_memory("Julia는 누구인가요?")

# 메모리 내용 표시
print("=== memory contents ===")
print(memory.load_memory_variables({}))

=== Stuff Documents Chain RAG pipeline with memory ===

1. Aaronson이 유죄인가?
Q: Aaronson은 유죄인가요?
A: 네, Jones, Aaronson 및 Rutherford는 그들이 기소된 범죄에 유죄로 판결받았습니다. 그러나 그들의 유죄를 반증하는 사진을 본 적이 없다고 기억합니다. 사실, 그런 사진은 존재하지 않았고, 유죄 판결을 뒷받침하는 증거를 그가 발명했다고 기억합니다.
--------------------------------------------------
2. 그가 테이블에 어떤 메시지를 썼나요?
Q: 그가 테이블에 어떤 메시지를 썼나요?
A: 그가 테이블에 썼던 메시지는 "2수 안에 백이 승리합니다"였습니다. 이것은 체스 문제를 의미하는 것으로, 그가 백색 말을 움직여서 2수 안에 상대를 승리시킬 수 있다는 것을 나타냅니다.
--------------------------------------------------
3. Julia는 누구인가요?
Q: Julia는 누구인가요?
A: Julia는 주인공인 윈스턴과 사랑을 나누는 여성 캐릭터입니다. 그들은 미래의 오세아니아에서 파티의 감시와 통제 속에서 금지된 사랑을 경험하게 됩니다. 그러나 이 사랑은 파티의 규칙을 어기는 것으로 간주되며 위험한 상황에 처하게 됩니다.
--------------------------------------------------
=== memory contents ===
{'chat_history': [HumanMessage(content='Aaronson은 유죄인가요?'), AIMessage(content='네, Jones, Aaronson 및 Rutherford는 그들이 기소된 범죄에 유죄로 판결받았습니다. 그러나 그들의 유죄를 반증하는 사진을 본 적이 없다고 기억합니다. 사실, 그런 사진은 존재하지 않았고, 유죄 판결을 뒷받침하는 증거를 그가 발명했다고 기억합니다.'), HumanMessage(content